In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import urllib.request
import numpy as np

urllib.request.urlretrieve(
    "https://github.com/google/fonts/raw/main/ofl/ibmplexmono/IBMPlexMono-Regular.ttf",
    "IBMPlexMono-Regular.ttf",
)
fe = font_manager.FontEntry(fname="IBMPlexMono-Regular.ttf", name="plexmono")
font_manager.fontManager.ttflist.append(fe)
plt.rcParams.update(
    {
        "axes.facecolor": "#f5f4e9",
        "grid.color": "#AAAAAA",
        "axes.edgecolor": "#333333",
        "figure.facecolor": "#FFFFFF",
        "axes.grid": False,
        "axes.prop_cycle": plt.cycler("color", plt.cm.Dark2.colors),
        "font.family": fe.name,
        "figure.figsize": (3.5, 3.5 / 1.2),
        "ytick.left": True,
        "xtick.bottom": True,
    }
)

import random

np.random.seed(0)
random.seed(0)

## Util functions to process Spectre information

In [ ]:
import numpy as np 
import pandas as pd
import os
# TODO: Refactor all these functions to take into account that we also have abs spectra now

def _get_peaks(x, y):
    peaks = []
    th = 0.001
    for i in range(1, len(x)-1):
        if y[i] > y[i-1]+th and y[i] > y[i+1]+th:
            peaks.append((x[i], y[i]))
    return peaks

def get_peaks(x, y):
    x = np.array(x)
    p1_i = np.where(x == 450)[0][0]
    p1_e = np.where(x == 550)[0][0]
    p2_i = np.where(x == 550)[0][0]
    p2_e = np.where(x == 800)[0][0]

    x1 = x[p1_i:p1_e]
    y1 = y[p1_i:p1_e]
    x2 = x[p2_i:p2_e]
    y2 = y[p2_i:p2_e]

    return x1[np.argmax(y1)], x2[np.argmax(y2)]

def get_area_under_peaks(x, y, norm=None):
    if not norm:
        return np.trapz(y, x)
    
    x = np.array(x)
    p1_i = np.where(x == 450)[0][0]
    p1_e = np.where(x == 550)[0][0]
    p2_i = np.where(x == 550)[0][0]
    p2_e = np.where(x == 800)[0][0]

    x1 = x[p1_i:p1_e]
    y1 = y[p1_i:p1_e]
    x2 = x[p2_i:p2_e]
    y2 = y[p2_i:p2_e]

    return np.trapz(y1, x1)/norm, np.trapz(y2, x2)/norm

def process_spectrum(x, y, norm_area):
    peaks = get_peaks(x, y)
    areas = get_area_under_peaks(x, y, norm_area)
    return peaks, areas

def smooth_spectrum(x, y, window=4):
    y = np.array(y)
    y_smooth = np.zeros_like(y)
    for i in range(len(y)):
        if i < window:
            y_smooth[i] = np.mean(y[:i+window])
        elif i > len(y) - window:
            y_smooth[i] = np.mean(y[i-window:])
        else:
            y_smooth[i] = np.mean(y[i-window:i+window])
    return x, y_smooth.tolist()

## Get f=AUC2*QY for new data

In [ ]:
#get list of files in Data/fluoressence_data starting in M
import os
files = os.listdir('Data/fluorescence_data/proposed_trials/BO/')
#files = os.listdir('Data/fluorescence_data/proposed_trials/NLP/')
#split files in two lists, one .txt and one .csv
txt_files = [file for file in files if file.endswith('.txt')]
csv_files = [file for file in files if file.endswith('.csv')]
#organize them together such that the first .txt file corresponds to the first .csv file
txt_files.sort() #To make sure they are zipped in the right order correctly
csv_files.sort() #To make sure they are zipped in the right order correctly

for txt, csv in zip(txt_files, csv_files):
    #with open(f"Data/fluorescence_data/proposed_trials/NLP/{txt}", 'r') as input:
    with open(f"Data/fluorescence_data/proposed_trials/BO/{txt}", 'r') as input:
                lines = input.readlines()
                x = []
                y = []
                for line in lines:
                    x.append(float(line.split()[0]))
                    y.append(float(line.split()[1]))
                y = [yi-min(y) for yi in y]
                x, y = smooth_spectrum(x, y)
                fl_int_norm = [yi/max(y) for yi in y]
                fl_auc_total = get_area_under_peaks(x, y)
                peaks, areas = process_spectrum(x, y, norm_area=fl_auc_total)
                fl_wvl, fl_int = x[:], y[:]
    
    #if os.path.exists(f"Data/fluorescence_data/proposed_trials/NLP/{csv}"):
    if os.path.exists(f"Data/fluorescence_data/proposed_trials/BO/{csv}"):
                df = pd.read_csv(f"Data/fluorescence_data/{csv}")
                df.sort_values(by=['nm'], inplace=True) # wavelengths were saved in reversed order
                x, y = df['nm'].astype(float).to_list(), df[' A'].astype(float).to_list()
                x, y = smooth_spectrum(x, y)
                abs_int_norm = [yi/max(y) for yi in y]
                abs_auc_total = get_area_under_peaks(x, y)
                abs_wvl, abs_int = x[:], y[:]
                # qy = qy_r * (fl_s/fl_r) * (abs_r/abs_s) * (n_s/n_r)**2 Constant values took from Farwa's spreadsheet
                abs_s = df[df['nm'] == 430][' A'].values[0]
                fl_r, abs_r, n_r = 8272505, 0.129734, 1.3611
                fl_s, abs_s, n_s = fl_auc_total, abs_s, 1.375
                qy = 0.53 * (fl_auc_total/fl_r) * (abs_r/abs_s) * (n_s/n_r)**2
                print(abs_s,"absorbance at 430",csv)

    print(txt,"QY:",qy,"AUC2:", areas[1],"f:", qy*areas[1])

## Processing raw data

In [ ]:
def f_(d):
    if d.is_integer():
        return f"{int(d)}"
    else:
        decimal = len(str(d).split('.')[1])
        s = f"{d:.{decimal}f}"
        return s
experiments = pd.read_csv('Data/experiments.csv')   # Total of 56 experiments of raw data
                                                    # some data is incomplete, and depending on the purpose of the analysis we might want to discard some of it (e.g.
                                                    # we dont have pre-doping absorbance so we cant calculate predoping QY (pre-QY) for some of the NPLs)
processed_df = pd.DataFrame()
for exp in experiments['Experiment'].unique():
    exp_df = experiments[experiments['Experiment'] == exp]

    for i, setup in exp_df[['Temperature(C)', 'Doping(%)', 'Time(min)']].iterrows():
        T, d, t = setup
        with open(f"Data/fluorescence_data/{exp}/{int(T)}-{f_(d)}-{int(t)}.txt", 'r') as input:
            lines = input.readlines()
            x = []
            y = []
            for line in lines:
                x.append(float(line.split()[0]))
                y.append(float(line.split()[1]))
            x, y = smooth_spectrum(x, y)
            #substract minimum value to all values
            y = [yi-min(y) for yi in y]
            fl_int_norm = [yi/max(y) for yi in y]
            fl_auc_total = get_area_under_peaks(x, y)
            peaks, areas = process_spectrum(x, y, norm_area=fl_auc_total)
            fl_wvl, fl_int = x[:], y[:]

        if os.path.exists(f"Data/absorbance_data/{exp}/{int(T)}-{f_(d)}-{int(t)}.csv"):
            df = pd.read_csv(f"Data/absorbance_data/{exp}/{int(T)}-{f_(d)}-{int(t)}.csv")
            df.sort_values(by=['nm'], inplace=True) # wavelengths were saved in reversed order
            x, y = df['nm'].astype(float).to_list(), df[' A'].astype(float).to_list()
            x, y = smooth_spectrum(x, y)
            abs_int_norm = [yi/max(y) for yi in y]
            abs_auc_total = get_area_under_peaks(x, y)
            abs_wvl, abs_int = x[:], y[:]
            # qy = qy_r * (fl_s/fl_r) * (abs_r/abs_s) * (n_s/n_r)**2 Constant values took from Farwa's spreadsheet
            abs_s = df[df['nm'] == 430][' A'].values[0]
            fl_r, abs_r, n_r = 8272505, 0.129734, 1.3611
            fl_s, abs_s, n_s = fl_auc_total, abs_s, 1.375
            qy = 0.53 * (fl_auc_total/fl_r) * (abs_r/abs_s) * (n_s/n_r)**2
            qypeak1 = 0.53 * (areas[0]*fl_auc_total/fl_r) * (abs_r/abs_s) * (n_s/n_r)**2
            qypeak2 = 0.53 * (areas[1]*fl_auc_total/fl_r) * (abs_r/abs_s) * (n_s/n_r)**2
        else:
            print(f"File not available: Data/absorbance_data/{exp}/{int(T)}-{f_(d)}-{int(t)}.csv")
            abs_int_norm = None
            abs_auc_total = None
            abs_wvl, abs_int = None, None
            qy = -1
        #if "pre doping" in folder 
        """Some of the NPLs where measured after synthesis/before doping """
        pre_fl_files = ["NPL-4-Predoping 100x dilution 3mL cuvette.txt",
                 "NPL-11 Pre Doping Stock-100x dilution (3mL).txt",
                 "NPL-17 100x Pre Doping.txt",
                 "NPL-17 100x Pre doping stock.txt",
                 "NPL-12 Pre Doping 100x dil.txt",
                 "NPL-13 Pre Doping 100x dil.txt"]
        pre_abs_files = ["NPL-4 100x Pre Doping .csv",
                         "NPL-11 Pre Doping Stock-100x dilution (3mL).csv",
                         "NPL-17 100x Pre Doping.csv",
                         "NPL-17 100x Pre doping stock.csv",
                         "NPL-12 1x Pre doping 100 x.Sample.Raw.csv",
                         "NPL-13 Pre doping 100 x dil .Sample.Raw.csv"]
        #iterate files in directory.
        for file in os.listdir(f"Data/fluorescence_data/{exp}"):
            if file in pre_fl_files:
                print(file)
                with open(f"Data/fluorescence_data/{exp}/{file}", 'r') as input:
                    lines = input.readlines()
                    x = []
                    y = []
                    for line in lines:
                        x.append(float(line.split()[0]))
                        y.append(float(line.split()[1]))
                    x, y = smooth_spectrum(x, y)

                    y = [yi-min(y) for yi in y]                      #substract minimum value to all values (baseline correction)
                    pre_fl_int_norm = [yi/max(y) for yi in y]
                    pre_fl_auc_total = get_area_under_peaks(x, y)
                    pre_peaks, pre_areas = process_spectrum(x, y, norm_area=fl_auc_total)
                    pre_fl_wvl, pre_fl_int = x[:], y[:]
        flag = 0
        for file in os.listdir(f"Data/absorbance_data/{exp}"):
            if file in pre_abs_files:
                flag = 1
                df = pd.read_csv(f"Data/absorbance_data/{exp}/{file}")
                df.sort_values(by=['nm'], inplace=True) # wavelengths were saved in reversed order
                x, y = df['nm'].astype(float).to_list(), df[' A'].astype(float).to_list()
                x, y = smooth_spectrum(x, y)
                abs_int_norm = [yi/max(y) for yi in y]
                abs_auc_total = get_area_under_peaks(x, y)
                abs_wvl, abs_int = x[:], y[:]
                # qy = qy_r * (fl_s/fl_r) * (abs_r/abs_s) * (n_s/n_r)**2 Constant values took from Farwa's spreadsheet
                pre_abs_s = df[df['nm'] == 430][' A'].values[0]
                pre_fl_r, pre_abs_r, n_r = 8272505, 0.129734, 1.3611
                pre_fl_s, pre_abs_s, n_s = pre_fl_auc_total, pre_abs_s, 1.375
                print(pre_fl_auc_total,pre_fl_r,pre_abs_r,pre_abs_s)
                pre_qy = 0.53 * (pre_fl_auc_total/pre_fl_r) * (pre_abs_r/pre_abs_s) * (n_s/n_r)**2
        if flag == 0:
                print(f"File not available: Data/absorbance_data/{exp}")
                abs_int_norm = None
                abs_auc_total = None
                abs_wvl, abs_int = None, None
                pre_qy = -1  #if we dont have pre-doping absorbance we cant calculate pre-doping QY (-1 is the flag to not use the data)
             
        #we add whatever we want to the results_df, we can add other labels here just be sure to calculate them
        #above and any new util function should be added to the top of the file
        results_df = pd.DataFrame({'Temperature(C)': [T], 
                                    'Doping(%)': [d], 
                                    'Time(min)': [t], 
                                    'Peak1': [peaks[0]], 
                                    'Peak2': [peaks[1]], 
                                    'Area1': [areas[0]], 
                                    'Area2': [areas[1]],
                                    'Pre-QY': [pre_qy],
                                    'QY': [qy],
                                    'QY_peak1': [qypeak1],
                                    'QY_peak2': [qypeak2],
                                    # 'Distance_peaks': [peaks[1]-peaks[0]],
                                    # 'ratioP2P1': [areas[1]/areas[0]],
                                    'Fl_Wavelengths': [fl_wvl],
                                    'Fluorescence': [fl_int],
                                    'Fluorescence_norm': [fl_int_norm],
                                    'Abs_Wavelengths': [abs_wvl],
                                    'Absorbance': [abs_int],
                                    'Absorbance_norm': [abs_int_norm]
                                    }) # we can add other labels here
        new_entry = exp_df.merge(results_df, how='inner', on=['Temperature(C)', 'Doping(%)', 'Time(min)'])
        processed_df = pd.concat([processed_df, new_entry], ignore_index=True)
processed_df.to_csv('Data/processed_datawobaselinev4.csv', index=False)

## Making data for Bayesian Optimization with GPs

In [ ]:
df = pd.read_csv('Data/processed_datawobaselinev4.csv')


#Maintain only Dimension1(nm),Dimension2(nm),OD,Dilution(%),Nano_stock_vol(mL),Diluted_vol(mL),AgConc(M),AgMass(mg),MeOH_vol(mL),H2O_vol(mL),Doping(%),AgSol(uL),Time(min),Temperature(C),Experiment,
grouped = df.groupby(['Experiment','Doping(%)', 'Time(min)', 'Temperature(C)'])
features_labels_dict = {}

for name, group in grouped:
    features = group[['Dimension1(nm)', 'Dimension2(nm)', 'Dilution(%)', 'Nano_stock_vol(mL)', 
                      'Diluted_vol(mL)', 'AgConc(M)', 'AgMass(mg)', 'MeOH_vol(mL)', 
                      'H2O_vol(mL)', 'Doping(%)', 'AgSol(uL)', 'Time(min)', 'Temperature(C)']] #we can add more features here or remove some of them
    
    label = group['Area2']
    label2 = group['QY']
    
    features_labels_dict[name] = {'features': features, 'label': label, 'labelqy': label2}
group_keys = list(features_labels_dict.keys())
#remove points where qy == -1

predictions = {}
# Get the group keys for training and testing
train_keys = [group_keys[i] for i in range(len(group_keys)) ]
test_keys = group_keys  # test_keys_index is a list with one elemen
# Prepare the training and testing data
train_features = np.vstack([features_labels_dict[key]['features'] for key in train_keys])
train_labels = np.hstack([features_labels_dict[key]['label'] for key in train_keys])
train_labels2 = np.hstack([features_labels_dict[key]['labelqy'] for key in train_keys])
#remove data with -1 in QY for both qy and area2/ -1 is our flag for missing data
train_labels = train_labels[train_labels2 != -1]  
train_features = train_features[train_labels2 != -1]
train_labels2 = train_labels2[train_labels2 != -1]
if len(train_labels) == len(train_labels2)== len(train_features):
    train_final_label = train_labels * train_labels2
else:
     print("this is not working")


modified_features = train_features.copy()
#only use the independent features
modified_features = modified_features[:,[0,1,2,3,5,6,9,11,12]] #TODO: Make it such that i can use the names for clarity.

#convert to tensor
#modified_features = torch.tensor(modified_features.values, dtype=torch.float)
#train_final_label = torch.tensor(train_final_label.values, dtype=torch.float)
#make sure to round up value up to two decimals
#save modified features and train_final_label to csv
df = pd.DataFrame(modified_features)
df['label'] = train_labels2
NAME_OF_FILE = 'Data/modified_featuresv4.csv'
df.to_csv(NAME_OF_FILE)
with open(f'{NAME_OF_FILE}_description.txt', 'a') as f:
    """Describe the columns of the modified features csv
    and what are you changing or wanting to analyze from this dataset"""
    description = """###ENTER DESCRIPTION HERE###
    ###Example: Dataset where i include pre-qy to the features. 
    ###Example: My hypothesis is that it will improve the model performance."""
    f.write(f"Final_label,{train_final_label}\n")
    f.write(description)

#analysis of the data